Information entropy:
$$
\mathrm{Ent}(D)=-\sum_{k=1}^{|\mathcal{Y}|}p_k\log_2p_k.
$$
$D:$ total sample data

$p_k:$ the percentage of the kst sample

Information gain
$$
\mathrm{Gain}(D,a)=\mathrm{Ent}(D)-\sum_{v=1}^V\frac{|D^v|}{|D|}\mathrm{Ent}(D^v).
$$
suppose $a$ (one attribute) has $V$ cases $(a^1,a^2,...a^V)$


For an example, this is the data set:
![](1.png)

The information entropy is:
$$
\mathrm{Ent}(D)=-\sum_{k=1}^{2}p_{k}\log_{2}p_{k}=-\left(\frac{8}{17}\log_{2}\frac{8}{17}+\frac{9}{17}\log_{2}\frac{9}{17}\right)=0.998.
$$
If we choose '色泽' to split this data, that means:
$$
D^1\text{(色泽=青绿)}， D^2 (色泽=乌黑)， D^3 ( 色泽=浅白).
$$
And the information gain are:
$$
\begin{gathered}
\mathrm{Ent}(D^{1}) =-\left(\frac36\log_2\frac36+\frac36\log_2\frac36\right)=1.000, \\
\mathrm{Ent}(D^{2}) =-\left(\frac46\log_2\frac46+\frac26\log_2\frac26\right)=0.918, \\
\mathrm{Ent}(D^3) =-\left(\frac15\log_2\frac15+\frac45\log_2\frac45\right)=0.722, 
\end{gathered}
$$
$$
\begin{aligned}
\mathrm{Gain}(D,\text{色泽})& =\mathrm{Ent}(D)-\sum_{v=1}^{3}\frac{|D^{v}|}{|D|}\mathrm{Ent}(D^{v})  \\
&=0.998-\left(\frac{6}{17}\times1.000+\frac{6}{17}\times0.918+\frac{5}{17}\times0.722\right) \\
&=0.109.
\end{aligned}
$$
By the same way,
$$
\begin{aligned}&\mathrm{Gain}(D,\text{根蒂})=0.143;\quad\mathrm{Gain}(D,\text{敲声})=0.141;\\&\mathrm{Gain}(D,\text{纹理})=0.381;\quad\mathrm{Gain}(D,\text{脐部})=0.289;\\&\mathrm{Gain}(D,\text{触感})=0.006.\end{aligned}
$$

“纹理” is the max one! So, we will have the follow dividing:
\begin{array}{cc}&\boxed{\text{纹理}=?}\\[5pt]\text{清晰}&\text{稍糊}&\text{模糊}\\\boxed{\{1,2,3,4,5,6,8,10,15\}}&\boxed{\{7,9,13,14,17\}}&\boxed{\{11,12,16\}}\end{array}

The next step is: for “清晰”, the attribute is “色泽，根蒂，敲声，脐部，触感”, and calculate the information gain:
$$
\begin{aligned}&\mathrm{Gain}(D^{1},\text{色泽})=0.043;\quad\mathrm{Gain}(D^{1},\text{根蒂})=0.458;\\&\mathrm{Gain}(D^{1},\text{敲声})=0.331;\quad\mathrm{Gain}(D^{1},\text{脐部})=0.458;\\&\mathrm{Gain}(D^{1},\text{触感})=0.458.\end{aligned}
$$

We will get the tree at last:
![](2.png )

In [579]:
import sys
sys.path.append('../dataset')
import data3
import numpy as np

_data, _labels, _labels_full  = data3.createDataSet()


In [580]:
""" 
modify data
"""
labels = _labels[:-2]
data = list()
for i in range(len(_data)):
    data.append(_data[i][0:-3]+[_data[i][-1]])

labels_full = _labels_full

In [581]:
""" 
entropy:
p1 means the percentage of good
p2 means the percentage of bad
"""
def entropy(p1, p2):
    if p1 == 0 or p2 == 0:
        entropy = -max(p1,p2)*np.log(max(p1,p2))
    else:
        entropy = -(p1*np.log(p1)+p2*np.log(p2))
    return entropy/ np.log(2)

In [582]:
""" 
get Ent(D)
"""
def origin_entropy(data):
    p1 = sum(1 for item in data if item[-1] == "好瓜")/\
        (sum(1 for item in data if item[-1] != "好瓜")+sum(1 for item in data if item[-1] == "好瓜"))
    p2 = 1 - p1
    return entropy(p1,p2)

In [583]:
""" 
counts the number of the good or bad of the ist attribute.
For example, if the ist attribute is ‘色泽’, 
we will count the number of good melon about ‘色泽=青绿’, 
              the number of bad melon about ‘色泽=青绿’, 
              the number of good melon about ‘色泽=浅白’,
              ..., .
This method is used by information_gain()
"""
def count(ist,data):
    labelList = [[data[ist]]+[data[-1]] for data in data]
    good_counts = {attribute: 0 for attribute in labels_full[labels[ist]]}
    bad_counts = {attribute: 0 for attribute in labels_full[labels[ist]]}
    for item in labelList:
        attribute, label = item
        if label == '好瓜' and attribute in labels_full[labels[ist]]:
            good_counts[attribute] += 1
        else:
            bad_counts[attribute]+=1
    return good_counts, bad_counts

""" 
information_gain:
ist means the ist attribute.
This will return the information gain of the ist attribute.
"""

def information_gain(last_entropy, ist,data):
    good, bad = count(ist,data)
    D = np.zeros(len(bad))
    total = sum(good.values())+sum(bad.values())
    for j, key in enumerate(good.keys()):
        num_good = good.get(key, 0)
        num_bad = bad.get(key, 0)
        if (num_good+num_bad) == 0:
            return 0
        p1 = num_good/(num_good+num_bad)
        p2 = 1-p1
        D[j] = (num_good+num_bad)/total*entropy(p1,p2)
    D = sum(D)
    return last_entropy-D

In [584]:
""" 
build the decision tree
"""
def best_feature_to_split(data):
    Gain = [information_gain(origin_entropy(data),i,data) for i in range(len(labels))]
    return Gain.index(max(Gain))

def build_tree(data):
    last_entropy = origin_entropy(data)
    Gain = [information_gain(last_entropy,i,data) for i in range(len(labels))]
    if max(Gain) == 0:
        return data[-1][-1]
    best_feature = best_feature_to_split(data)
    tree = {labels[best_feature]:{}}
    unique_values = labels_full[labels[best_feature]]
    for value in unique_values:
        sub_index = [index for index, row in enumerate(data) if value in row]
        sub_data = [data[i] for i in sub_index]
        sub_tree = build_tree(sub_data)
        tree[labels[best_feature]][value] = sub_tree
    return tree 

In [585]:
build_tree(data)

{'纹理': {'稍糊': {'触感': {'软粘': '好瓜', '硬滑': '坏瓜'}},
  '清晰': {'根蒂': {'稍蜷': {'触感': {'软粘': '坏瓜', '硬滑': '好瓜'}},
    '硬挺': '坏瓜',
    '蜷缩': '好瓜'}},
  '模糊': '坏瓜'}}